In [17]:
# read overlay 

from pynq import Overlay
overlay = Overlay("fft.bit")
LOGN = 10
N = 2**LOGN

In [18]:
dma_DATA = overlay.axi_dma_0
dma_CONFIG = overlay.axi_dma_1

In [19]:
import numpy as np
from pynq import allocate

In [20]:
dataIN = allocate(shape=(N,),dtype='uint32')
dataOUT = allocate(shape=(N,),dtype='int32')
config = allocate(shape=(1,), dtype='uint32')

In [21]:
fftdata = np.random.randint(50, size=(N,), dtype=np.uint32)
np.copyto(dataIN, fftdata)
np.copyto(config, np.array([0b000001110000000000000000 + LOGN], dtype='uint32'))

In [22]:
dma_CONFIG.sendchannel.transfer(config)
dma_CONFIG.sendchannel.wait()

In [23]:
%%timeit
#dma_CONFIG.sendchannel.transfer(config)
dma_DATA.sendchannel.transfer(dataIN)
dma_DATA.recvchannel.transfer(dataOUT)
#dma_CONFIG.sendchannel.wait()
dma_DATA.sendchannel.wait()
dma_DATA.recvchannel.wait()
#dataOUT

692 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%%timeit 
npres = np.fft.fft(fftdata)

385 µs ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
